# Loading libaries

In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from scipy import stats
import matplotlib.pyplot as plt
from copy import deepcopy
from plotly.subplots import make_subplots

# Anomaly

## Loading Data

In [2]:
DIR = 'Results/Catch22/'

In [3]:
df_real_1 = pd.read_csv(DIR+'features_df_signal_1_real_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('real_1_')
df_real_2 = pd.read_csv(DIR+'features_df_signal_2_real_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('real_2_')

df_generated_without_CD_1 = pd.read_csv(DIR+'Without_CD/features_df_signal_1_generated_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('generated_without_CD_1_')
df_generated_without_CD_2 = pd.read_csv(DIR+'Without_CD/features_df_signal_2_generated_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('generated_without_CD_2_')

df_generated_with_CD_1 = pd.read_csv(DIR+'With_CD/features_df_signal_1_generated_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('generated_with_CD_1_')
df_generated_with_CD_2 = pd.read_csv(DIR+'With_CD/features_df_signal_2_generated_data_frame_sine_with_anomaly_multiple_22.csv', index_col=0).add_prefix('generated_with_CD_2_')

df_real_1

,real_1_DN_HistogramMode_5,real_1_DN_HistogramMode_10,real_1_CO_f1ecac,real_1_CO_FirstMin_ac,real_1_CO_HistogramAMI_even_2_5,real_1_CO_trev_1_num,real_1_MD_hrv_classic_pnn40,real_1_SB_BinaryStats_mean_longstretch1,real_1_SB_TransitionMatrix_3ac_sumdiagcov,real_1_PD_PeriodicityWang_th0_01,...,real_1_DN_OutlierInclude_p_001_mdrmd,real_1_DN_OutlierInclude_n_001_mdrmd,real_1_SP_Summaries_welch_rect_area_5_1,real_1_SB_BinaryStats_diff_longstretch0,real_1_SB_MotifThree_quantile_hh,real_1_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,real_1_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,real_1_SP_Summaries_welch_rect_centroid,real_1_FC_LocalSimple_mean3_stderr,real_1_ID
0,-0.029871,-0.296276,40.0,2.0,0.557761,-0.008395,0.931164,91.0,0.029586,151.0,...,-0.366250,-0.042500,0.912501,6.0,1.816882,0.127660,0.872340,0.03068,0.384749,1.0
1,-0.097240,-0.316493,42.0,96.0,0.953182,0.001229,0.867334,91.0,0.015779,193.0,...,-0.370000,0.145000,0.975791,9.0,1.592702,0.851064,0.702128,0.03068,0.207935,2.0
2,-0.118359,-0.363338,41.0,1.0,0.824677,0.005412,0.919900,83.0,0.035503,158.0,...,-0.366875,0.135625,0.946469,8.0,1.738476,0.872340,0.702128,0.03068,0.299492,1.0
3,-0.034404,-0.269543,40.0,8.0,0.661733,-0.004315,0.929912,90.0,0.021696,168.0,...,-0.366250,-0.021250,0.923262,5.0,1.780599,0.872340,0.702128,0.03068,0.362935,1.0
4,-0.262852,-0.046972,41.0,2.0,0.780720,-0.000938,0.901126,91.0,0.005917,152.0,...,-0.368750,0.323750,0.949940,7.0,1.728974,0.872340,0.702128,0.03068,0.291176,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.054444,-0.299022,39.0,2.0,0.525561,-0.025928,0.938673,82.0,0.021696,154.0,...,-0.370000,0.175000,0.890168,5.0,1.820954,0.148936,0.872340,0.03068,0.432837,1.0
996,-0.158585,0.047994,41.0,1.0,1.041493,-0.000613,0.866083,91.0,0.009862,200.0,...,-0.373750,0.152500,0.976298,6.0,1.629320,0.851064,0.702128,0.03068,0.203496,2.0
997,-0.163463,0.065616,41.0,96.0,0.884135,0.000157,0.869837,92.0,0.017751,179.0,...,-0.376875,0.367500,0.965016,7.0,1.642491,0.851064,0.702128,0.03068,0.246218,2.0
998,-0.179536,0.083562,40.0,1.0,0.535223,0.008805,0.949937,87.0,0.009862,159.0,...,-0.360000,0.455000,0.895659,5.0,1.846056,0.127660,0.872340,0.03068,0.413253,1.0


## Correlation Analysis

### Computing Correlation

In [4]:
real_corr = pd.concat([df_real_1.drop(columns=["real_1_ID"]), df_real_2.drop(columns=["real_2_ID"])], axis=1).corr()
real_corr = real_corr[list(df_real_2.columns)[:-1]].loc[list(df_real_1.columns)[:-1]]

generated_without_CD = pd.concat([df_generated_without_CD_1, df_generated_without_CD_2], axis=1).corr()
generated_without_CD = generated_without_CD[list(df_generated_without_CD_2.columns)].loc[list(df_generated_without_CD_1.columns)]

generated_with_CD = pd.concat([df_generated_with_CD_1, df_generated_with_CD_2], axis=1).corr()
generated_with_CD = generated_with_CD[list(df_generated_with_CD_2.columns)].loc[list(df_generated_with_CD_1.columns)]

indices_real = real_corr.columns[real_corr.isna().all()].tolist()
indices_generated_without_CD = generated_without_CD.columns[generated_without_CD.isna().all()].tolist()
indices_generated_with_CD = generated_with_CD.columns[generated_with_CD.isna().all()].tolist()

indices_total = indices_real + indices_generated_without_CD + indices_generated_with_CD

features = ['SP_Summaries_welch_rect_centroid', 'IN_AutoMutualInfoStats_40_gaussian_fmmi']
real_features_2 = ['real_2_'+f for f in features]
generated_without_CD_features_2 = ['generated_without_CD_2_'+f for f in features]
generated_with_CD_features_2 = ['generated_with_CD_2_'+f for f in features]

real_features_1 = ['real_1_'+f for f in features]
generated_without_CD_features_1 = ['generated_without_CD_1_'+f for f in features]
generated_with_CD_features_1 = ['generated_with_CD_1_'+f for f in features]

real_not_nan = real_corr.drop(real_features_2, axis=1)
generated_with_CD_not_nan = generated_with_CD.drop(generated_with_CD_features_2, axis=1)
generated_without_CD_not_nan = generated_without_CD.drop(generated_without_CD_features_2, axis=1)

real_not_nan = real_not_nan.drop(real_features_1, axis=0)
generated_with_CD_not_nan = generated_with_CD_not_nan.drop(generated_with_CD_features_1, axis=0)
generated_without_CD_not_nan = generated_without_CD_not_nan.drop(generated_without_CD_features_1, axis=0)

In [5]:
generated_without_CD_not_nan

,generated_without_CD_2_DN_HistogramMode_5,generated_without_CD_2_DN_HistogramMode_10,generated_without_CD_2_CO_f1ecac,generated_without_CD_2_CO_FirstMin_ac,generated_without_CD_2_CO_HistogramAMI_even_2_5,generated_without_CD_2_CO_trev_1_num,generated_without_CD_2_MD_hrv_classic_pnn40,generated_without_CD_2_SB_BinaryStats_mean_longstretch1,generated_without_CD_2_SB_TransitionMatrix_3ac_sumdiagcov,generated_without_CD_2_PD_PeriodicityWang_th0_01,generated_without_CD_2_CO_Embed2_Dist_tau_d_expfit_meandiff,generated_without_CD_2_FC_LocalSimple_mean1_tauresrat,generated_without_CD_2_DN_OutlierInclude_p_001_mdrmd,generated_without_CD_2_DN_OutlierInclude_n_001_mdrmd,generated_without_CD_2_SP_Summaries_welch_rect_area_5_1,generated_without_CD_2_SB_BinaryStats_diff_longstretch0,generated_without_CD_2_SB_MotifThree_quantile_hh,generated_without_CD_2_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,generated_without_CD_2_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,generated_without_CD_2_FC_LocalSimple_mean3_stderr
generated_without_CD_1_DN_HistogramMode_5,0.028037,-0.005652,-0.012886,-0.060756,-0.017014,-0.026946,0.020905,0.015327,-0.003847,0.013596,0.034189,-0.013488,-0.002823,-0.058408,-0.020416,-0.030932,0.007942,-0.028196,0.023369,0.014752
generated_without_CD_1_DN_HistogramMode_10,-0.029501,-0.002532,0.038963,0.021682,-0.031018,-0.028411,0.054494,0.004603,0.054741,-0.020752,-0.039292,0.037228,-0.026062,0.023488,-0.012443,-0.009240,0.040205,0.052036,-0.013401,0.021070
generated_without_CD_1_CO_f1ecac,0.002250,-0.004668,0.009413,0.028384,0.055709,0.030617,-0.030834,-0.008155,-0.010247,-0.034575,0.020117,0.025343,-0.059299,0.036239,0.060342,0.017360,-0.051442,-0.001232,-0.024482,-0.059939
generated_without_CD_1_CO_FirstMin_ac,-0.007796,0.003185,0.026732,0.020743,0.042141,0.011284,-0.009177,-0.072214,-0.001899,-0.020840,0.056212,0.011836,-0.025944,-0.023368,0.043675,-0.020126,-0.046492,-0.028688,0.027494,-0.044449
generated_without_CD_1_CO_HistogramAMI_even_2_5,-0.012257,-0.001875,-0.003341,0.017402,0.062784,0.066835,-0.020722,-0.016794,-0.007449,-0.018226,0.032504,0.027346,-0.020073,0.002269,0.054592,0.016264,-0.044053,-0.028037,-0.021986,-0.053625
generated_without_CD_1_CO_trev_1_num,0.020691,-0.013202,0.017177,0.000018,0.004334,-0.049735,0.002860,0.069107,0.028004,0.012583,0.020613,-0.052744,0.056049,-0.003060,0.011048,-0.044933,-0.017531,-0.035879,0.013760,-0.010668
generated_without_CD_1_MD_hrv_classic_pnn40,0.021005,0.012689,-0.014114,-0.016377,-0.091535,-0.063276,0.038585,0.037992,0.004620,-0.013330,-0.057429,-0.021425,0.031222,-0.017311,-0.075762,-0.019889,0.061205,0.050016,0.024684,0.075861
generated_without_CD_1_SB_BinaryStats_mean_longstretch1,-0.025504,-0.002118,-0.006228,0.013490,0.048471,0.041149,-0.025377,0.008106,-0.005878,-0.027362,0.001868,0.027337,-0.068902,0.025321,0.035617,0.005731,-0.039536,-0.022108,-0.011292,-0.034700
generated_without_CD_1_SB_TransitionMatrix_3ac_sumdiagcov,-0.014070,-0.007900,0.032654,0.016616,0.062704,0.070490,-0.041706,0.024926,-0.040829,-0.001984,0.058802,0.006173,-0.029056,0.033244,0.067485,-0.016170,-0.026583,-0.023849,-0.049029,-0.063650
generated_without_CD_1_PD_PeriodicityWang_th0_01,0.002554,-0.003809,-0.006197,0.009473,0.025113,0.025650,0.005274,0.020373,0.003439,-0.027166,0.008657,0.023779,-0.048010,-0.027752,0.030048,0.010288,-0.020554,0.000060,-0.003589,-0.029280


### Plotting Correlation

In [6]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Real Data',
                                    'Without CD', 'With CD'))

fig.add_trace(
    px.imshow(real_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=1
)
fig.add_trace(
    px.imshow(generated_without_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=2
)

fig.add_trace(
    px.imshow(generated_with_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=3
)
fig.update_xaxes(showticklabels=False) 
fig.update_yaxes(showticklabels=False) 
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 1)
fig.update_yaxes(title_text="Feature Channel 2", row = 1, col = 1)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 2)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 3)

fig.update_layout(height=240, width=800, margin = {'l':0,'r':0,'t':23,'b':0},)
fig.show()

In [8]:
fig.write_image("Sine_anomaly_correlation.pdf")

### Spearman's rho

In [9]:
SPR_without_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
SPR_with_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("SPR for Signal 1 without CD: ", SPR_without_CD_1)
print("SPR for Signal 1 with CD: ", SPR_with_CD_1)

SPR for Signal 1 without CD:  SpearmanrResult(correlation=0.4274539215870099, pvalue=3.37476892402985e-19)
SPR for Signal 1 with CD:  SpearmanrResult(correlation=0.7862355389721184, pvalue=3.151205545666241e-85)


### Kendall's Tau

In [10]:
KT_without_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
KT_with_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("KT for Signal 1 without CD: : ", KT_without_CD_1)
print("KT for Signal 1 with CD: : ", KT_with_CD_1)

KT for Signal 1 without CD: :  KendalltauResult(correlation=0.2970175438596492, pvalue=7.203743024559329e-19)
KT for Signal 1 with CD: :  KendalltauResult(correlation=0.6117042606516291, pvalue=1.4041788260284364e-74)


### Frobenius Norm

In [11]:
FBNorm_without_CD_1 = np.linalg.norm(np.array(real_not_nan) - np.array(generated_without_CD_not_nan))
FBNorm_with_CD_1 = np.linalg.norm(np.array(real_not_nan )- np.array(generated_with_CD_not_nan))

print("Frobenius Signal for Real 1 without CD: ", FBNorm_without_CD_1)
print("Frobenius Signal for Real 1 with CD: ", FBNorm_with_CD_1)

Frobenius Signal for Real 1 without CD:  8.113104249964131
Frobenius Signal for Real 1 with CD:  5.361870802350751


### MAE

In [12]:
MSE_without_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_without_CD_not_nan)))
MSE_with_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_with_CD_not_nan)))

print("MSE of Real Data 1 and Generated Data without CD 1: ", MSE_without_CD_1)
print("MSE of Real Data 1 and Generated Data with CD 1: ", MSE_with_CD_1)

MSE of Real Data 1 and Generated Data without CD 1:  0.28897325136339896
MSE of Real Data 1 and Generated Data with CD 1:  0.19865122637281288
